In [16]:
import pandas as pd
import pickle
import numpy as np
from datetime import timedelta
from datetime import datetime

Sitzungssummary

In [17]:
sitzungen = pd.read_csv('../00_data/raw/sitzungssummary.csv')
names = ['ID','UserID','UserAttribut','AufgabenID','Version','TestID','Art','HA','Fehler','Altdatum','Datum','Saetze','Korrektur','isExperiment']
sitzungen.columns = names

# Filter 
sitzungen = sitzungen.loc[sitzungen['UserAttribut'] == 'Schüler']
sitzungen = sitzungen.loc[sitzungen['Art'] == 'GK']
sitzungen = sitzungen.drop(columns=['Altdatum'])
sitzungen = sitzungen[(sitzungen['Datum'] > '2022-06-21') & (sitzungen['Datum'] < '2022-10-31')]
sitzungen = sitzungen.loc[sitzungen['isExperiment']==1]
sitzungen = sitzungen.rename(columns={"ID": "UebungsID"})

sitzungen

,UebungsID,UserID,UserAttribut,AufgabenID,Version,TestID,Art,HA,Fehler,Datum,Saetze,Korrektur,isExperiment
9576,29905708,933218,Schüler,73,1,1913437,GK,nt,2,2022-06-21 00:02:29,b,1,1
9590,29896434,958267,Schüler,52,1,1955381,GK,vt,1,2022-06-21 00:12:56,b,0,1
9599,29896436,958267,Schüler,73,2,1955381,GK,vt,3,2022-06-21 00:24:56,b,0,1
9600,29905746,975835,Schüler,132,3,1990778,GK,nt,0,2022-06-21 00:25:25,b,0,1
9605,29896435,958267,Schüler,110,2,1955381,GK,vt,3,2022-06-21 00:31:45,b,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
811663,30716811,1031124,Schüler,91,1,2103679,GK,vt,9,2022-10-30 23:42:20,b,0,1
811664,30600394,1031479,Schüler,4,1,2104099,GK,zt,12,2022-10-30 23:47:00,b,1,1
811666,30716277,1031482,Schüler,69,2,2104102,GK,zt,0,2022-10-30 23:50:57,b,1,1
811667,30716812,1031124,Schüler,14,1,2103679,GK,vt,11,2022-10-30 23:53:38,b,0,1


XML Saetze

In [18]:
xml = pd.read_csv('../00_data/raw/xmlsaetze.csv')
names = ['ID','UserID','UebungsID','UserAttribut','Testart','Testposition','SatzID','Erstloesung','Schussel','KorrekturBenutzt','Datum','Erfolg','XMLstring','Loesungsnr']
xml.columns = names

# Filter 
xml = xml.loc[xml['UserAttribut'] == 'Schüler']
xml = xml.loc[xml['Testart'] == 'GK']
xml = xml[(xml['Datum'] > '2022-06-21') & (xml['Datum'] < '2022-10-31')]

xml['Loesungsnr']=xml['Loesungsnr'].str.split()
xml['MehrfachFalsch'] = xml['Loesungsnr'].str.len()
xml['MehrfachFalsch'] = xml['MehrfachFalsch'].replace([np.nan],0)
xml['MehrfachFalsch'] = xml['MehrfachFalsch'].astype('int')
xml['MehrfachFalsch'] = xml['MehrfachFalsch'] - 1



Saetze

In [19]:
saetze = pd.read_csv('../00_data/raw/saetze.csv')
names = ['satzID','AufgabenID','Art','Satznr','Versionsnr','Aufgabe','Loesung','Hilfe','Erklaerung','Schwierigkeit','Gesamtschwierigkeit','XMLstring']
saetze.columns = names
#Filter 
saetze = saetze.loc[saetze['Art'] == 'GK']

Schueler

In [20]:
schueler = pd.read_csv('../00_data/raw/schueler.csv')
names = ['ID','Name','Passwort_express','Email','Loginname','Passwort','SessionID','Lehrername','LehrerID','Klassenname','Geschlecht','Klassenstufe','Anmeldedatum','Anmeldeklassenstufe','Altklasse','gesperrt','Aufgaben','Altaufgaben','done','Info','interventiongroup']
schueler.columns = names
schueler = schueler[['ID','Geschlecht','Klassenstufe','interventiongroup']]
schueler = schueler.rename(columns={"ID": "UserID"})
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 6),'6',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 2),'2',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 3),'3',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 5),'5',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == '0'),'control',schueler.interventiongroup)


C:\Users\Nathalie\AppData\Local\Temp\ipykernel_4980\3544100862.py:1: DtypeWarning: Columns (1,2,3,6,7,15,17,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  schueler = pd.read_csv('../00_data/raw/schueler.csv')


Predictions

In [21]:
pred = pd.read_csv('../00_data/raw/predictions.csv')
names = ['ID','UebungsID','SatzID','UserID','interventiongroup','prediction','seqMode','modus','versionline','version','Datum']
pred.columns = names
pred['Datum'] = pd.to_datetime(pred['Datum'].astype(str)) + pd.DateOffset(hours=2)
pred = pred[(pred['Datum'] > '2022-06-21') & (pred['Datum'] < '2022-10-31')]
pred['Date'] = pd.to_datetime(pred['Datum']).dt.date
pred['Time'] = pd.to_datetime(pred['Datum']).dt.time
pred['Minute'] = pd.to_datetime(pred['Datum']).dt.minute
pred['interventiongroup'] = np.where((pred.interventiongroup == 6),'6',pred.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 2),'2',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 3),'3',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 5),'5',schueler.interventiongroup)
pred['interventiongroup'] = np.where((pred.interventiongroup == '0'),'control',pred.interventiongroup)


Merging and Filtering

In [22]:
#only keep records with users who are in sitzungen (which is filtered by isExperiment)
userlist = sitzungen.UserID.unique()
xml = xml.loc[xml['UserID'].isin(userlist)]
schueler = schueler.loc[schueler['UserID'].isin(userlist)]
valid = ['m','w','d']
schueler = schueler.loc[schueler['Geschlecht'].isin(valid)]
valid = ['9', '8', '10', '7', '5', '12', '6', '11', '13']
schueler = schueler.loc[schueler['Klassenstufe'].isin(valid)]

In [23]:
# filter xml by uebungsIDs
uebungslist = sitzungen.UebungsID.unique()
xml = xml.loc[xml['UebungsID'].isin(uebungslist)]


In [24]:
sitzungen = pd.merge(sitzungen, schueler, on='UserID')

Save files

In [25]:
sitzungen.to_pickle('../00_data/cleaned/sitzungssummary.pkl')
pred.to_pickle('../00_data/cleaned/predictions.pkl')
saetze.to_pickle('../00_data/cleaned/saetze.pkl')
xml.to_pickle('../00_data/cleaned/xmlsaetze.pkl')
